## Useful functions required in the program:

**1)** For converting 2D array into 1D list

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

**2)** for Converting a list into a string (for easy to read print statements)

In [ ]:
def list_to_string(a):  # for printing a list as single string
  sam = ''
  for i in a:
    sam = sam+', '+i
  return sam

# Connecting to DB

In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
import pandas as pd
import numpy as np

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'my_json_file.json'  #link to json file

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

records_file = 'your_records_googlesheet_id'  # record file google sheet id
keywords_file = 'your_keywords_googlesheet_id'  # keywords file google sheet id

In [ ]:
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
records_rows = sheet.values().get(spreadsheetId=records_file, range='A1:K1').execute()  # whenever new category gets updated, this line has to be changed
keyword_rows = sheet.values().get(spreadsheetId=keywords_file, range='A:B').execute()

records = flatten(records_rows['values'])
keywords = pd.DataFrame(data=keyword_rows.get('values', []))
keywords = keywords.rename(columns=keywords.iloc[0]).drop(keywords.index[0])

# Taking Input

In [15]:
input = """medical test 1000  # enter keywords followed by expense value. for muliple entries use next line
burger 78"""  

In [24]:
sentences = input.split('\n')
expenses = dict()
text,integer = list(),list()
for i in sentences:
  a = i.split(' ')
  text_sub = ''
  for j in a:
    try:
      integer.append(int(j))
    except:
      text_sub = text_sub+j+' '
  text.append(text_sub.rstrip())
expenses = {text[i]: integer[i] for i in range(len(text))}

# Text processing

### correcting irregularities in data

In [ ]:
keywords.dropna(subset=['Keywords'], inplace=True)

In [ ]:
keywords['Category'] = keywords['Category'].str.replace('""','')

In [ ]:
# categories in consideration
categories = records[1:]

## Accepting new entries by user

In [ ]:
import builtins
unique_keys = list() # unique list of keys
unique_categories = list()  # unique list of categories
for i in expenses.keys():
  if i not in list(keywords['Keywords']):
    unique_keys.append(i)

In [ ]:
# if we get new entries, lets take its category to be placed into
if len(unique_keys) >= 1:
  print("These are the unique keys: ",unique_keys)
  print("Categories:",list_to_string(categories))
  print("What will be their categories? (add double space in between): ")
  unique_categories.append(builtins.input().split('  '))
if unique_categories:
  unique_categories = flatten(unique_categories)

## Taking Update frequency

In [ ]:
# update frequency is the number of days for which we want our data to entered in records.
import builtins
freq_day = int(builtins.input("Enter number of days after you're updating the data (press 1 for today): "))

Enter number of days after you're updating the data (press 1 for today): 1


## Formatting Date

Generating list of Dates for the no. of days records needs to be added

In [ ]:
from datetime import datetime, timedelta 
start = datetime.today().date()
end = (datetime.today() - timedelta(days=freq_day)).date()
date_generated = list()
for i in range(0, freq_day):
  b = start - timedelta(days=i)
  date_generated.append(b.strftime('%d-%m-%Y'))

Creating single dictionary containing category and expense

In [ ]:
# first extracting details from unique items
category_vals = dict()
for i in expenses.keys():
  for j,v in enumerate(unique_keys):
    if i == v:
      try:
        category_vals[unique_categories[j]] = category_vals[unique_categories[j]] + expenses[i]
      except:
        category_vals[unique_categories[j]] = expenses[i]

In [ ]:
#for general items
for i,v in expenses.items():
  if i not in unique_keys:
    try:
      category_vals[keywords[keywords['Keywords'] == i]['Category'].iloc[0]] = category_vals[keywords[keywords['Keywords'] == i]['Category'].iloc[0]] + v
    except:
      category_vals[keywords[keywords['Keywords'] == i]['Category'].iloc[0]] =  v

Dividing the values amonst days

In [ ]:
for i,j in category_vals.items():
  category_vals[i] = j/freq_day

# Creating Dataframes

For Keywords

In [ ]:
keywords_final = pd.DataFrame({'Keywords':unique_keys,'Category':unique_categories})

For Records

In [ ]:
row = [0] * len(categories)

In [ ]:
for i, v in enumerate(categories):
  for j in category_vals.keys():
    if v == j:
      row[i] = category_vals[j]

In [ ]:
category_final = pd.DataFrame()
category_final['date'] = date_generated
category_final[categories] = row

# Pushing Values to GSheets

In [ ]:
request_records = sheet.values().append(spreadsheetId=records_file, range='A:J', valueInputOption= "USER_ENTERED", insertDataOption= "INSERT_ROWS", body= {"values": category_final.values.tolist()}).execute()

In [ ]:
if len(keywords_final) >=1:
  request_keywords = sheet.values().append(spreadsheetId=keywords_file, range='A:B', valueInputOption= "USER_ENTERED", insertDataOption= "INSERT_ROWS", body= {"values": keywords_final.values.tolist()}).execute()